In [76]:
"""Generate a random pandas dataframe with two numerical columns and one categorical column"""
import random
import pandas as pd
import numpy as np

def generate_data(n):
    """Generate a random pandas dataframe with two numerical columns and one categorical column"""
    data = []
    for i in range(n):
        """Add data to a list"""
        data.append([random.random(), random.random(), random.choice(['0','1','2'])])
        # testNum = random.random()
        # data.append([testNum, testNum+0.1*random.random(), if(testNum > 0.5, "A", "B")])
    """Convert list to numpy array"""
    data = np.array(data)
    """Convert list of lists to pandas dataframe"""
    data = pd.DataFrame(data, columns=['x1', 'x2', 'y'])
    return data

my_data = generate_data(100)
my_data.head

<bound method NDFrame.head of                      x1                     x2  y
0    0.8384432184232014     0.9274954924754585  1
1    0.5175615803602975     0.8056419517796346  0
2    0.3074148074861729     0.8010486520029023  0
3   0.34883556190048126     0.4898641810711224  1
4   0.08843834856046884     0.4227741901111495  0
..                  ...                    ... ..
95  0.14533969704262029    0.07515080780777073  1
96   0.2998190819621035  0.0018495128437687747  2
97  0.19677914200163782     0.4761490597463197  0
98  0.43043592164136324     0.1700810911124574  0
99   0.9407099851212435     0.3091743127881722  2

[100 rows x 3 columns]>

In [77]:
"""Convert columns to categorical"""
my_data['y'] = my_data['y'].astype('category')

"""Convert columsn to numeric"""
my_data['x1'] = my_data['x1'].astype('float')
my_data['x2'] = my_data['x2'].astype('float')

my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   x1      100 non-null    float64 
 1   x2      100 non-null    float64 
 2   y       100 non-null    category
dtypes: category(1), float64(2)
memory usage: 1.9 KB


In [78]:
""" Build a classifier using the pandas dataframe """
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def train_model(df):
    """Trains a model using columns of df"""
    """Split data into training and test sets"""
    train, test = train_test_split(df, test_size=0.2)
    """Create a decision tree classifier"""
    clf = DecisionTreeClassifier()
    """Train the classifier"""
    clf.fit(train[['x1', 'x2']], train['y'])
    return clf, train, test

def test_model(clf, test):
    """Predict the labels of the test set"""
    pred = clf.predict(test[['x1', 'x2']])
    """Calculate the accuracy of the model"""
    accuracy = accuracy_score(test['y'], pred)
    return accuracy

In [79]:
clf, train, test  = train_model(my_data)
# accuracy = test_model(clf, test)

"""Evaluate model performance using classification_report"""
accuracy = classification_report(test['y'], clf.predict(test[['x1', 'x2']]))

print(accuracy)

              precision    recall  f1-score   support

           0       0.43      0.43      0.43         7
           1       0.40      0.67      0.50         6
           2       0.33      0.14      0.20         7

    accuracy                           0.40        20
   macro avg       0.39      0.41      0.38        20
weighted avg       0.39      0.40      0.37        20



In [84]:
"""Train a keras neural network to classify the data"""
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.models import load_model


def train_keras_model(df):
    """Split data into training and test sets"""
    df = my_data.copy()
    train, test = train_test_split(df, test_size=0.2)
    """Convert y to categorical"""
    train['y'] = to_categorical(train['y'])
    test['y'] = to_categorical(test['y'])
    """Create a neural network"""
    model = Sequential()
    """Add layers to the neural network"""
    model.add(Dense(10, input_dim=2, activation='relu'))
    model.add(Dense(1, activation='softmax'))  #3 because there are 3 classes in the response variable
    """Compile the neural network"""
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    """Train the neural network"""
    model.fit(train[['x1', 'x2']], train['y'], epochs=100, batch_size=10, verbose=1)
    """Evaluate the model"""
    score = model.evaluate(test[['x1', 'x2']], test['y'], verbose=0)
    return model, train, test, score

In [85]:
 model, train, test, score = train_keras_model(my_data)  
 score

Epoch 1/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 3/100
8/8 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 4/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 6/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 7/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 8/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 9/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3125
Epoch 10/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accura

[0.0, 0.25]

In [86]:
"""Save the model"""
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


In [87]:
""" Load the model"""
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
"""Another way to Save the model """
model.save("model_allinOne.h5")
print("Saved model to disk")

"""Another way to Load the model """
model = load_model("model_allinOne.h5")
print("Loaded model from disk")

In [89]:
"""Predict the labels of the test set"""
pred = model.predict(test[['x1', 'x2']])

"""Evaluate model performance using classification_report"""
accuracy = classification_report(test['y'], pred)

print(accuracy)


1/1 [==============================] - 0s 23ms/step
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        15
         1.0       0.25      1.00      0.40         5

    accuracy                           0.25        20
   macro avg       0.12      0.50      0.20        20
weighted avg       0.06      0.25      0.10        20



/Users/pgmenon/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pgmenon/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pgmenon/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
